In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from notebook_prelude import *

In [ ]:
np.random.seed(42)

IMAGE_FOLDER = 'tmp/permutation_test'
EXTENSIONS = ['png', 'pdf']
FONT_SIZE = 10
CMAP = 'tab10'
GRID_COLOR = '#CCCCCC'
FIGSIZE = (2.5, 6)

colors =plt.get_cmap('Paired').colors
arrow_color = 'red'
model_a_color, model_b_color = colors[0], colors[1]
model_colors = [model_a_color, model_b_color]

os.makedirs(IMAGE_FOLDER, exist_ok=True)

def save_fig(fig, filename_without_ext, folder = IMAGE_FOLDER, extensions = EXTENSIONS):
    for ext in extensions:
        filename = '{}/{}.{}'.format(folder, filename_without_ext, ext)
        fig.savefig(filename)

In [ ]:

def cleanup_axes(ax):
    ax.grid('off')
    for pos, spine in ax.spines.items(): spine.set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
def draw_rect(x, y, width, height, ax, fill = False, edgecolor="none", linewidth=2, **rect_kwargs):
    ax.add_patch(
        matplotlib.patches.Rectangle(
            (x, y),   # (x,y)
            width,          # width
            height,          # height
            fill=fill,
            edgecolor=edgecolor,
            linewidth=linewidth,
            **rect_kwargs
        )
    )


def add_text(**kwargs):
    ax.text(**kwargs, fontdict=dict(horizontalalignment='center', verticalalignment='center'))


def draw_permutation_box(texts, ax, scores, difference, model_colors, permutations, arrow_color, x_off = 0.35, y_off =0.01, box_w = 0.6, box_h = 0.9, grid_color = GRID_COLOR, left_text_offset_x = 0.12, arrow_x_offset = 0.63, draw_doc_labels = True):
    box_max_x, box_max_y = x_off + box_w, y_off + box_h
    # + scores + difference
    num_boxes = len(texts) + 2
    box_height = box_h / num_boxes
    cell_width = box_w / 2
    
    # Draw boxes
    for i, text in enumerate(texts):
        i = i + 2
        y = (i * box_height) + y_off
        real_index = len(texts) - i + 1
        is_permutated = real_index in permutations
        offset = 1 if is_permutated else 0
        color_a = model_colors[offset]
        color_b = model_colors[(offset + 1) % 2]
        draw_rect(x_off, y, cell_width, box_height, ax = ax, fill = True, color = color_a)
        draw_rect(x_off + cell_width, y, cell_width, box_height, ax = ax, fill = True, color = color_b)
    
    def add_text_to_cell(cell_w, cell_h, text = 'Yes'):
        cell_x = x_off + ((0.5 + cell_w) * cell_width)
        
        cell_y = y_off + (box_height * (cell_h + 0.5))
        add_text(x = cell_x, y = cell_y, s = text)
    
    # Draw outside rect
    draw_rect(x_off, y_off, box_w, box_h, ax, edgecolor=grid_color)
    
    # Middle line
    ax.axvline(x = (box_w / 2) + x_off - 0.01, ymin=y_off + box_height, ymax=box_max_y, color = grid_color)
    
    # Add horizontal cell lines
    for i in range(num_boxes - 1):
        y_start = y_off + (box_height * (i + 1))
        ax.axhline(y = y_start, xmin = x_off, xmax = box_max_x, color = grid_color)

    # Just a little thicker line
    y_start = y_off + box_height * 1 - 0.002
    ax.axhline(y = y_start, xmin = x_off, xmax = box_max_x, color = grid_color)
    y_start = y_off + box_height * 2 - 0.002
    ax.axhline(y = y_start, xmin = x_off, xmax = box_max_x, color = grid_color)

    # Model predictions
    for i, (doc_text, cell_a, cell_b) in enumerate(reversed(texts + [scores])):
        add_text_to_cell(0, i + 1, cell_a)
        add_text_to_cell(1, i + 1, cell_b)
    
    # Last rows
    add_text(x = left_text_offset_x, y = y_off + (1.5 * box_height), s = 'Score')
    add_text(x = left_text_offset_x, y = y_off + (0.5 * box_height), s = 'Difference')
    
    # Difference text
    add_text(x = x_off + 1 * cell_width, y = y_off + (0.5 * box_height), s = difference)
    
    # Model names
    add_text(x = x_off + 0.5 * cell_width, y = box_h + y_off + 0.03, s = 'Model\nA')
    add_text(x = x_off + 1.5 * cell_width, y = box_h + y_off + 0.03, s = 'Model\nB')
    
    # Permutation arrows
    for box in permutations:
        y = (num_boxes - box)
        cell_y = y_off + (box_height * y) - box_height / 2
        ax.arrow(arrow_x_offset, cell_y, 0.05, 0, head_width=0.01, head_length=0.02, linewidth=2, shape = 'full', edgecolor = arrow_color)
        ax.arrow(arrow_x_offset + 0.02, cell_y, -0.05, 0, head_width=0.01, head_length=0.02, linewidth=2, shape = 'full', edgecolor = arrow_color)
    
    # Document texts
    if draw_doc_labels:
        for i, (doc_text, _, _) in enumerate(reversed(texts)):
            add_text(x = left_text_offset_x, y = y_off + ((i + 2 + 0.5) * box_height), s = doc_text)

    fig.tight_layout()

FIG_w, FIG_h = FIGSIZE
fig, axes = plt.subplots(figsize = (FIG_w * 3, FIG_h), ncols=3)
for i, ax in enumerate(axes):
    cleanup_axes(ax)
    texts = [
        ('Doc #1', 'A', 'B'),
        ('Doc #2', 'A', 'B'),
        ('Doc #3', 'A', 'B'),
        ('Doc #...', '...', '...'),
        ('Doc #n-2', 'A', 'B'),
        ('Doc #n-1', 'A', 'B'),
        ('Doc #n', 'A', 'B'),
    ]

    arrow_preds = [0, 1, 2, 5]
    scores = ('', 0.98, 0.98)
    draw_permutation_box(texts = texts, ax = ax, permutations = arrow_preds, scores = scores, difference = 0, model_colors = model_colors, arrow_color = arrow_color, draw_doc_labels = i == 0)


In [ ]:
fig, ax = plt.subplots(figsize = FIGSIZE)
def draw_true_predictions_box(predictions, x_off = 0.45, y_off =0.1, box_w = 0.6 / 2, box_h = 0.9, grid_color = GRID_COLOR, ax = None, left_text_offset_x = 0.18):
    num_boxes = len(predictions)
    num_normal = num_boxes + 2
    missing_boxes = num_normal - num_boxes
    box_height_ = box_h * (num_boxes / num_normal)
    y_off = box_h - box_height_
    # Draw outside rect
    draw_rect(x_off, y_off, box_w, box_height_, ax, edgecolor=grid_color)
    cell_height = box_height_ / num_boxes
    cell_width = box_w
    
    def add_text_to_cell(cell_w, cell_h, text = 'Yes'):
        cell_x = x_off + ((0.5 + cell_w) * cell_width)
        cell_y = y_off + (cell_height * (cell_h + 0.5))
        add_text(x = cell_x, y = cell_y, s = text)
    
    for i, (label, pred) in enumerate(reversed(predictions)):
        y = cell_height * (i + missing_boxes)
        ax.axhline(y = y, xmin = x_off, xmax = x_off + cell_width, color = grid_color)
        add_text_to_cell(0, i, pred)
        add_text(x = left_text_offset_x, y = y_off + ((i + 0.5) * cell_height), s = label)
    
    add_text(x = x_off + 0.5 * cell_width, y = box_height_ + y_off + 0.04, s = 'True Labels')


In [ ]:
def get_model_and_preds(num_elements = 1000, labels = ['A', 'B', 'C']):
    preds = np.zeros((3, num_elements), dtype=str)
    #preds = [np.array([]), np.array([]), np.array([])]
    for i in range(num_elements):
        preds[0][i] = np.random.choice(labels)
        preds[1][i] = np.random.choice(labels)
        preds[2][i] = np.random.choice(labels)
    return preds

In [ ]:
def get_text_from(*preds, num_before = 3, num_after = 3):
    out = []
    for i in range(num_before):
        o = ['Doc #{} '.format(i)]
        for pred in preds:
            o.append(pred[i])
        out.append(o)
    len_elements = len(preds)
    out.append(['Doc #...'] + (['...'] * len_elements))
    for i in range(num_after):
        o = ['Doc #n{}'.format(-num_after + i + 1 if i != num_after -1 else '')]
        for pred in preds:
            o.append(pred[i])
        out.append(o)
    return out

In [ ]:
y_true, y_pred_a, y_pred_b = get_model_and_preds()

def get_score_and_difference(y_true, y_pred_a, y_pred_b, metric = sklearn.metrics.accuracy_score):
    score_a = metric(y_true = y_true, y_pred = y_pred_a)
    score_b = metric(y_true = y_true, y_pred = y_pred_b)
    return score_a, score_b, score_a - score_b

def permutate(x, y):
    assert len(x) == len(y)
    perm_mat = np.random.randint(0, 2, len(x), dtype=np.uint)
    out_x, out_y = [], []
    for perm, val_x, val_y in zip(perm_mat, x, y):
        if perm == 0:
            out_x.append(val_x)
            out_y.append(val_y)
        else:
            out_x.append(val_y)
            out_y.append(val_x)
    return out_x, out_y, perm_mat

out = []
initial_score_a, initial_score_b, initial_diff = get_score_and_difference(y_true, y_pred_a, y_pred_b)
for i in range(1000):
    y_pred_a_mixed, y_pred_b_mixed, permut_mat = permutate(y_pred_a, y_pred_b)
    out.append((y_pred_a_mixed, y_pred_b_mixed, permut_mat, get_score_and_difference(y_true, y_pred_a_mixed, y_pred_b_mixed)))
    #permutated_labels.append((y_pred_a_mixed, y_pred_b_mixed))
    #score_a, score_b, difference = get_score_and_difference(y_true, y_pred_a_mixed, y_pred_b_mixed)
    #diffs.append(difference)

#fig, ax = plt.subplots()
#pd.DataFrame(diffs, columns = ['diffs']).plot(kind='hist', bins = 30, ax = ax)
#save_fig(fig, 'distribution')

In [ ]:
fig, ax = plt.subplots(figsize = FIGSIZE)
texts = get_text_from(y_true)
draw_true_predictions_box(texts, ax = ax)
cleanup_axes(ax)
save_fig(fig, 'true_labels')


In [ ]:
fig, ax = plt.subplots(figsize = FIGSIZE)
cleanup_axes(ax)
scores = ['', initial_score_a, initial_score_b]
draw_permutation_box(texts = get_text_from(y_pred_a, y_pred_b), ax = ax, permutations = [], scores = scores, difference = initial_diff, model_colors = model_colors, arrow_color = arrow_color, draw_doc_labels = True)
save_fig(fig, 'initial_predictions')

In [ ]:
NUM_SAMPLES = 3
NUM_BEFORE = 3
NUM_AFTER = 3

fig, axes = plt.subplots(figsize = (FIG_w * NUM_SAMPLES, FIG_h), ncols=NUM_SAMPLES)
choices = np.random.choice(len(out), NUM_SAMPLES)
choices = [0, 2, 4]
choice = np.array(out, dtype=object)[choices]
for idx, (ax, (y_pred_a_mixed, y_pred_b_mixed, permut_mat, (score_a, score_b, difference))) in enumerate(zip(axes, choice)):
    cleanup_axes(ax)
    permuted_indices = np.where(permut_mat == 1)[0]
    texts = get_text_from(y_pred_a_mixed, y_pred_b_mixed, num_before= NUM_BEFORE, num_after=NUM_AFTER)
    permutations = permuted_indices[(permuted_indices < NUM_BEFORE) | (permuted_indices > len(y_pred_a_mixed) - NUM_AFTER)]
    permutations[permutations > NUM_BEFORE] += NUM_BEFORE + 4 - len(y_pred_a_mixed)
    scores = ['', score_a, score_b]
    draw_permutation_box(texts = texts, ax = ax, permutations = permutations, scores = scores, difference = difference, model_colors = model_colors, arrow_color = arrow_color, draw_doc_labels = idx == 0)
    fig.tight_layout()

save_fig(fig, 'samples')

In [ ]:
diffs = [difference for _, _, _, (_, _, difference) in out]

fig, ax = plt.subplots(figsize = (13, 3))
df_diffs = pd.DataFrame(diffs, columns = ['diffs'])
min_, max_ = df_diffs.diffs.min(), df_diffs.diffs.max()
df_diffs.diffs.plot(kind='hist', bins = 60, normed = True, ax = ax)
ax.axvline(x=initial_diff, color='red')
ax.axvline(x=-initial_diff, color='red')


# Blue area
draw_rect(-np.fabs(initial_diff), 0, np.fabs(initial_diff) * 2, 1000, ax=ax, fill=True, facecolor ='blue', edgecolor=None, alpha = 0.1)

draw_rect(min_, 0, np.fabs(min_ - initial_diff), 1000, ax=ax, fill='red', facecolor ='red', edgecolor=None, alpha = 0.1)
draw_rect(np.fabs(initial_diff), 0, np.fabs(max_ - initial_diff), 1000, ax=ax, fill='red', facecolor ='red', edgecolor=None, alpha = 0.05)

ax.set_xlim(min_, max_)
fig.tight_layout()
save_fig(fig, 'distribution')

In [ ]:
num_samples = len(diffs)
confidence = len(np.where(np.fabs(diffs) > np.fabs(initial_diff))[0]) / num_samples
confidence